In [0]:
%sql
-- 📊 Cálculo da média de valor total recebido por mês
-- Esta consulta calcula a média de 'total_amount' para cada partição (anomes)
-- utilizando a camada Silver que criamos (jan–mai/2023).
-- O resultado mostra, para cada mês, o valor médio recebido por corrida.

SELECT 
    anomes,
    ROUND(AVG(total_amount), 2) AS media_total_amount
FROM workspace.nyc_taxi.yellowtaxi_trips_2023_spec
GROUP BY anomes
ORDER BY anomes;


In [0]:
%sql
-- 📊 Cálculo da média de passageiros por hora do dia (Mês de Maio/2023)
-- Passos:
-- 1. Filtra a camada Silver para incluir apenas registros de maio de 2023 (anomes = '202305').
-- 2. Extrai a hora do dia a partir do campo de pickup (tpep_pickup_datetime).
-- 3. Calcula a média de passageiros por hora.
-- 4. Ordena o resultado de 0h a 23h para facilitar a análise temporal.

SELECT 
    HOUR(tpep_pickup_datetime) AS hora_do_dia,
    ROUND(AVG(passenger_count), 2) AS media_passageiros
FROM workspace.nyc_taxi.yellowtaxi_trips_2023_spec
WHERE anomes = '202305'
GROUP BY hora_do_dia
ORDER BY hora_do_dia;


In [0]:
from pyspark.sql import functions as F
import matplotlib.pyplot as plt

# 📥 1. Consulta agregada para o mês de maio/2023
df_maio = (
    spark.table("workspace.nyc_taxi.yellowtaxi_trips_2023_spec")
    .filter(F.col("anomes") == "202305")
    .groupBy(F.hour("tpep_pickup_datetime").alias("hora_do_dia"))
    .agg(F.round(F.avg("passenger_count"), 2).alias("media_passageiros"))
    .orderBy("hora_do_dia")
)

# Converte o DataFrame Spark para Pandas para facilitar a plotagem no Matplotlib
pdf = df_maio.toPandas()

# 📊 2. Criação do gráfico de linha
# - Eixo X: horas do dia (0 a 23)
# - Eixo Y: média de passageiros por corrida
# - Inclui marcadores para destacar pontos
plt.figure(figsize=(10,5))
plt.plot(pdf["hora_do_dia"], pdf["media_passageiros"], marker="o")
plt.title("Média de Passageiros por Hora - Maio/2023")
plt.xlabel("Hora do Dia")
plt.ylabel("Média de Passageiros")
plt.grid(True)
plt.xticks(range(0, 24))
plt.show()
